# Coin flipping Bayes tutorial

This is a simple example of Bayesian estimation that demonstrates an approach to estimating the probability $p$ that a coin will come up heads after observing a series of flips of the coin.

History:

- 6/2006:   dhb  Wrote it, starting with some code that Barry Wark and I wrote for something else.
- 06/2010: RK revision
- 03/2020: MLW Python translation

Suppose someone is tossing a coin, and this coin has a probability $p$ of coming up heads on each flip. We don't know $p$. We get a series of observations of whether the coin comes up heads or tails, and we want to use Bayes rule to update our estimate $\hat p$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
p_true = .3

First step. Define a prior on $p$. To make things simple, we'll describe our prior on a discrete set of possibilities that $p$ might take on. We'll just start with a prior that is flat over the range $0$–$1$. You can play with different choices by setting this to something else

In [ ]:
n_bins = 100
possible_p_values = np.linspace(0, 1, n_bins)
prior_p_probs = np.ones(n_bins) / n_bins

Compute the prior mean, which we might tiake as out initial estimate of $p$:

In [ ]:
prior_mean = np.sum(prior_p_probs * possible_p_values)
prior_mean

Now we will estimate $\hat p$ from data.

Each time we loop, we'll observe a random coin toss (driven by the true probability $p$ specified above), compute the posterior using Bayes rule, and take the mean of the posterior to get our current estimate $\hat p$ of $p$. Then we'll update the prior by substituting in the posterior, and repeat.



In [ ]:
n_observations = 100
index = np.arange(n_observations)
observations = np.full(n_observations, np.nan)
posterior_p_probs = np.full(n_observations, np.nan)
p_est = np.full(n_observations, np.nan)

seed = sum(map(ord, "bayes tutorial"))
rng = np.random.default_rng(seed)

To show how the posterior evolves in real time, we'll define a matplotlib animation. First, we set up the figure that we want to see.

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(8, 4))

axes[0].set(
    xlim=(0, 1),
    ylim=(0, .2),
    xlabel="Possible values of $p$",
    ylabel="Probability",
)

axes[1].set(
    xlim=(0, n_observations),
    ylim=(-.05, 1.05),
    xlabel="Observation",
    ylabel="$\hat p$",
)

# Show the true value of p
axes[0].axvline(p_true, ls="--", color=".5")
axes[1].axhline(p_true, ls="--", color=".5")

# Draw the prior
axes[0].plot(possible_p_values, prior_p_probs)

# For the remaining plots, keep a reference to the line so we can update it

# Draw the posterior
posterior_plot, = axes[0].plot(possible_p_values, posterior_p_probs)

# Draw the observations
observation_plot, = axes[1].plot(index, observations, ls="", marker="x")

# Draw the estimate of p
p_est_plot, = axes[1].plot(index, p_est)

f.tight_layout()

Now define a function for updating the posterior on each observation:

In [ ]:
def update(i):

    # Observe a "flip"
    observations[i] = heads = rng.binomial(1, p_true)

    # Compute the likelihood of the coin coming up heads
    likelihood = possible_p_values if heads else 1 - possible_p_values

    # The unnormalized posterior is the prior times the likelihood
    posterior_p_probs = prior_p_probs * likelihood

    # The normalized posterior should sum to 1
    posterior_p_probs /= posterior_p_probs.sum()

    # The current estimate is given by the posterior mean
    p_est[i] = np.sum(posterior_p_probs * possible_p_values)

    # The posterior now becomes the prior for the next iteration
    prior_p_probs[:] = posterior_p_probs
    
    # Now update the plot
    posterior_plot.set_ydata(posterior_p_probs)
    observation_plot.set_ydata(observations)
    p_est_plot.set_ydata(p_est)

    return posterior_plot, observation_plot, p_est_plot

In [ ]:
anim = animation.FuncAnimation(f, update, n_observations)
HTML(anim.to_html5_video())